# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [0]:
# import libraries
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.decomposition import TruncatedSVD
import pickle

In [0]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# load data from database
engine = create_engine('sqlite:////content/disaster_dataset.db')
df = pd.read_sql_table("disaster_dataset", engine) 
#df.to_sql('table', myeng, index=False)

In [0]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [0]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [0]:
#test tokenize function
test = 'Hello I see fire in the street and many houses are destroyed, homeless people everywhere'
tokenize(test)

['hello',
 'i',
 'see',
 'fire',
 'in',
 'the',
 'street',
 'and',
 'many',
 'house',
 'are',
 'destroyed',
 ',',
 'homeless',
 'people',
 'everywhere']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [0]:
model = Pipeline([
       ('vect', CountVectorizer(tokenizer=tokenize)),
       ('tfidf', TfidfTransformer()),
       ('clf', MultiOutputClassifier(RandomForestClassifier())),
   ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [0]:
#create a function to easily assess a model's performance.
results_dict = {}

def perf_report(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report, f-1 score, overall accuracy
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print('***')
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))
        results_dict[col] = classification_report(y_test[col], y_pred[:, i], output_dict=True)
    # Convert to df
    weighted_avg = {}
    for key in results_dict.keys():
        weighted_avg[key] = results_dict[key]['weighted avg']

    df_w_avg = pd.DataFrame(weighted_avg).transpose()
    
    overall_accuracy = (y_pred == y_test).mean().mean()
    print('overall accuracy {0:.4f} \n'.format(overall_accuracy))
    print('f1-score {0:.4f} \n'.format(df_w_avg['f1-score'].mean()))
    
    return df_w_avg

In [0]:
perf_report(model, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.73      0.26      0.38      1266
           1       0.80      0.97      0.87      3938
           2       0.75      0.07      0.14        40

    accuracy                           0.79      5244
   macro avg       0.76      0.43      0.46      5244
weighted avg       0.78      0.79      0.75      5244

***
request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.87      0.41      0.56       895

    accuracy                           0.89      5244
   macro avg       0.88      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4649
           1       0.91      0.37      0.52       595

    accuracy                           0.92      5244
   macro avg       0.92      0.68      0.74      5244
weighted avg       0.92      0.92      0.91      5244

***
shelter
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4774
           1       0.88      0.24      0.38       470

    accuracy                           0.93      5244
   macro avg       0.90      0.62      0.67      5244
weighted avg       0.93      0.93      0.91      5244

***
clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       1.00      0.05      0.10        73

    accuracy                           0.99      5244
   macro avg       0.99      0.53      0.55      5244
weighted avg       0.99      0.99      0.98      

,precision,recall,f1-score,support
related,0.779244,0.790236,0.749207,5244.0
request,0.887895,0.889397,0.872337,5244.0
offer,0.990108,0.995042,0.992569,5244.0
aid_related,0.774537,0.772883,0.766545,5244.0
medical_help,0.904051,0.922197,0.891514,5244.0
medical_products,0.938072,0.950038,0.929115,5244.0
search_and_rescue,0.968182,0.976163,0.965794,5244.0
security,0.975291,0.983219,0.975271,5244.0
military,0.956061,0.970252,0.957556,5244.0
child_alone,1.000000,1.000000,1.000000,5244.0


### 6. Improve your model
Use grid search to find better parameters. 

In [0]:
model.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0

In [0]:
make_scorer(f1_score, average='micro')

parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(model, param_grid=parameters)

In [0]:
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [0]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [0]:
y_pred = cv.predict(X_test)

In [0]:
perf_report(cv, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.73      0.27      0.40      1266
           1       0.80      0.97      0.88      3938
           2       0.75      0.07      0.14        40

    accuracy                           0.79      5244
   macro avg       0.76      0.44      0.47      5244
weighted avg       0.78      0.79      0.75      5244

***
request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.88      0.42      0.57       895

    accuracy                           0.89      5244
   macro avg       0.89      0.71      0.76      5244
weighted avg       0.89      0.89      0.88      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4649
           1       0.91      0.36      0.51       595

    accuracy                           0.92      5244
   macro avg       0.92      0.68      0.74      5244
weighted avg       0.92      0.92      0.91      5244

***
shelter
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4774
           1       0.91      0.22      0.35       470

    accuracy                           0.93      5244
   macro avg       0.92      0.61      0.66      5244
weighted avg       0.93      0.93      0.91      5244

***
clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       1.00      0.04      0.08        73

    accuracy                           0.99      5244
   macro avg       0.99      0.52      0.54      5244
weighted avg       0.99      0.99      0.98      

,precision,recall,f1-score,support
related,0.782004,0.792906,0.754184,5244.0
request,0.890864,0.891876,0.875647,5244.0
offer,0.990108,0.995042,0.992569,5244.0
aid_related,0.777450,0.775172,0.768606,5244.0
medical_help,0.901438,0.921815,0.891282,5244.0
medical_products,0.934195,0.949466,0.928143,5244.0
search_and_rescue,0.969373,0.976354,0.966230,5244.0
security,0.971090,0.982838,0.975072,5244.0
military,0.959057,0.970633,0.957769,5244.0
child_alone,1.000000,1.000000,1.000000,5244.0


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [0]:
#Improve the model
new_model = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [0]:
new_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
                              random_state=None, tol=0.0)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
              

In [0]:
y_pred = new_model.predict(X_test)

In [0]:
perf_report(new_model, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.50      0.00      0.00      1266
           1       0.75      1.00      0.86      3938
           2       0.86      0.15      0.26        40

    accuracy                           0.75      5244
   macro avg       0.70      0.38      0.37      5244
weighted avg       0.69      0.75      0.65      5244

***
request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4349
           1       0.00      0.00      0.00       895

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.69      0.83      0.75      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***
shelter
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4774
           1       0.00      0.00      0.00       470

    accuracy                           0.91      5244
   macro avg       0.46      0.50      0.48      5244
weighted avg       0.83      0.91      0.87      5244

***
clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       0.00      0.00      0.00        73

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

***
money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5140
           1       0.00      0.00      0.00       104

    accuracy                           0.98      5244
   macro avg       0.49      0.50      0.49      5244
weighted avg       0.96      0.98      

,precision,recall,f1-score,support
related,0.691861,0.751907,0.646775,5244.0
request,0.687759,0.829138,0.751860,5244.0
offer,0.990108,0.994851,0.992474,5244.0
aid_related,0.587346,0.594394,0.447722,5244.0
medical_help,0.845530,0.919527,0.880977,5244.0
medical_products,0.899676,0.948513,0.923449,5244.0
search_and_rescue,0.952150,0.975782,0.963821,5244.0
security,0.966719,0.983219,0.974899,5244.0
military,0.941758,0.970442,0.955885,5244.0
child_alone,1.000000,1.000000,1.000000,5244.0


In [0]:
make_scorer(f1_score, average='micro')

new_parameters = { 'vect__ngram_range': ((1, 1), (1, 2)), 
              'vect__max_df': (0.5, 1.0), 
              'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [0]:
new_cv = GridSearchCV(new_model, param_grid=new_parameters)
new_cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [0]:
new_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [0]:
y_pred = new_cv.predict(X_test)

In [0]:
perf_report(new_cv, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.50      0.00      0.00      1266
           1       0.75      1.00      0.86      3938
           2       0.71      0.25      0.37        40

    accuracy                           0.75      5244
   macro avg       0.66      0.42      0.41      5244
weighted avg       0.69      0.75      0.65      5244

***
request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4349
           1       0.00      0.00      0.00       895

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.69      0.83      0.75      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***
shelter
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4774
           1       0.00      0.00      0.00       470

    accuracy                           0.91      5244
   macro avg       0.46      0.50      0.48      5244
weighted avg       0.83      0.91      0.87      5244

***
clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       0.00      0.00      0.00        73

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

***
money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5140
           1       0.00      0.00      0.00       104

    accuracy                           0.98      5244
   macro avg       0.49      0.50      0.49      5244
weighted avg       0.96      0.98      

,precision,recall,f1-score,support
related,0.691240,0.752288,0.647817,5244.0
request,0.687759,0.829138,0.751860,5244.0
offer,0.990108,0.994851,0.992474,5244.0
aid_related,0.587346,0.594394,0.447722,5244.0
medical_help,0.845530,0.919527,0.880977,5244.0
medical_products,0.899667,0.948322,0.923354,5244.0
search_and_rescue,0.952150,0.975782,0.963821,5244.0
security,0.966719,0.983219,0.974899,5244.0
military,0.941758,0.970442,0.955885,5244.0
child_alone,1.000000,1.000000,1.000000,5244.0


### 9. Export your model as a pickle file

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Udacity DS nanodegree/4.Data Enegeneering/Disaster Response Pipeline/model.pkl'
with open(path, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.